In [1]:
import json
import requests
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

D:\Programme\Anaconda\Neuer Ordner\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [6]:
# GLOBAL

coins = ["BTC", "XRP", "ETH", "DOGE",  "LTC", "BTB", "BITCNY", "XMR", "BLOCK", "XWC",
        "DCR", "NXT", "FCT", "DASH", "MAID", "BCN", "BTS", "SC", "DGB", "MONA"]


#coins = ['BCH', 'XLM']


days_ago_to_fetch = 1500  # see also filter_history_by_date()
coin_history = {}
hist_length = 0
average_returns = {}
cumulative_returns = {}

def fetch_all():
  for coin in coins:
    coin_history[coin] = fetch_history(coin)

def fetch_history(coin):
  endpoint_url = "https://min-api.cryptocompare.com/data/histoday?fsym={}&tsym=USD&limit={:d}".format(coin, days_ago_to_fetch)
  res = requests.get(endpoint_url)
  hist = pd.DataFrame(json.loads(res.content)['Data'])
  hist = index_history(hist)
  hist = filter_history_by_date(hist)
  return hist

def index_history(hist):
  # index by date so we can easily filter by a given timeframe
  hist = hist.set_index('time')
  hist.index = pd.to_datetime(hist.index, unit='s')
  return hist

def filter_history_by_date(hist):
  result = hist # customize here
  result = hist[hist.index.year >= 2016]
  # result = result[result.index.day == 1] # every first of month, etc.
  return result

fetch_all()
hist_length = len(coin_history[coins[0]])

coin_history['BCH'].head()

close  high  low  open  volumefrom  volumeto
time                                                    
2016-01-01    0.0   0.0  0.0   0.0         0.0       0.0
2016-01-02    0.0   0.0  0.0   0.0         0.0       0.0
2016-01-03    0.0   0.0  0.0   0.0         0.0       0.0
2016-01-04    0.0   0.0  0.0   0.0         0.0       0.0
2016-01-05    0.0   0.0  0.0   0.0         0.0       0.0

In [3]:
# Calculate returns and excess returns

def add_all_returns():
  for coin in coins:
    hist = coin_history[coin]
    hist['return'] = (hist['close'] - hist['open']) / hist['open']
    average = hist["return"].mean()
    average_returns[coin] = average
    cumulative_returns[coin] = (hist["return"] + 1).prod() - 1
    hist['excess_return'] = hist['return'] - average
    coin_history[coin] = hist

add_all_returns()

# display data
print(cumulative_returns)
coin_history['BTC'].head()

{'BTC': 7.041183429325972, 'XRP': 41.10831092405868, 'ETH': 97.14599478270235, 'DOGE': 173.69109134263212, 'LTC': 6.381968643090818, 'BTB': -0.8267896054919492, 'BITCNY': 7.164853883386023, 'XMR': 89.0757442749604, 'BLOCK': 3.8791949275137103, 'XWC': 19.592850870910745, 'DCR': 0.25299634686030226, 'NXT': -0.46055805035179975, 'FCT': 2.659292094079886, 'DASH': 138.09829386001584, 'MAID': 0.29466516467110826, 'BCN': 2.2492202957595606, 'BTS': 0.3337185701476124, 'SC': 18.879250173203143, 'DGB': 7.293973926987393, 'MONA': 8.137073157205425}


close    high     low    open  volumefrom     volumeto    return  \
time                                                                            
2016-01-01  433.99  437.95  425.86  429.95    46973.12  20334286.58  0.009396   
2016-01-02  433.72  437.43  430.73  433.99    33570.39  14566820.95 -0.000622   
2016-01-03  430.70  434.14  423.09  433.72    54826.26  23490162.61 -0.006963   
2016-01-04  433.32  435.26  428.60  430.70    53007.87  22943968.22  0.006083   
2016-01-05  431.20  435.25  428.87  433.32    45029.87  19499588.99 -0.004892   

            excess_return  
time                       
2016-01-01       0.006617  
2016-01-02      -0.003402  
2016-01-03      -0.009743  
2016-01-04       0.003303  
2016-01-05      -0.007672

In [4]:
# Excess matrix

excess_matrix = np.zeros((hist_length, len(coins)))

for i in range(0, hist_length):
  for idx, coin in enumerate(coins):
    # Matrix for absolute crypto-values
    excess_matrix[i][idx] = coin_history[coin].iloc[i]['close']
    # Matrix for simple returns
    #excess_matrix[i][idx] = coin_history[coin].iloc[i]['return']

# Display

pretty_matrix = pd.DataFrame(excess_matrix, columns = coins)
pretty_matrix.index = coin_history[coins[0]].index

pretty_matrix.head()

BTC       XRP     ETH      DOGE   LTC   BTB  BITCNY     XMR  \
time                                                                         
2016-01-01  433.99  0.005500  0.9200  0.000180  3.49  1.39  0.1894  0.5431   
2016-01-02  433.72  0.005125  0.9547  0.000169  3.49  1.43  0.1893  0.4600   
2016-01-03  430.70  0.005200  0.9600  0.000169  3.47  1.62  0.1880  0.4875   
2016-01-04  433.32  0.005100  0.9500  0.000149  3.49  1.43  0.1891  0.4700   
2016-01-05  431.20  0.005000  0.9426  0.000177  3.46  1.42  0.1882  0.4700   

              BLOCK       XWC     DCR       NXT     FCT  DASH     MAID  \
time                                                                     
2016-01-01  0.03060  0.000182  0.1085  0.006245  0.5863  3.18  0.01440   
2016-01-02  0.03611  0.000182  0.1084  0.006488  0.6705  3.00  0.01472   
2016-01-03  0.03456  0.000228  0.1076  0.006193  0.6413  2.80  0.01506   
2016-01-04  0.03510  0.000204  0.1083  0.006339  0.5997  2.70  0.01554   
2016-01-05  0.03262  0.000220  0.1078  0.006334  0.6101  2.96  0.01532   

                 BCN       BTS        SC       DGB     MONA  
time                                                         
2016-01-01  0.000026  0.003468  0.000022  0.000130  0.06011  
2016-01-02  0.000026  0.003431  0.000022  0.000130  0.06072  
2016-01-03  0.000030  0.003403  0.000022  0.000129  0.06025  
2016-01-04  0.000030  0.003380  0.000017  0.000126  0.06790  
2016-01-05  0.000034  0.003363  0.000017  0.000134  0.06822

In [50]:
 pretty_matrix.isnull().values.sum()

0

In [7]:
len(pretty_matrix.columns)

20

In [8]:
# Save Matrix
pretty_matrix.to_csv("crypto_20_test.csv")